In [3]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format("../models/180/model.bin", binary=True)

/home/akvarats/dev/ai/ada_datamasters_2019/classification-appeals/.venv/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

stop_words = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
    'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
    'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
    'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
    'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их',
    'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда',
    'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти',
    'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при',
    'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про',
    'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой',
    'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю',
    'между',
    'быть', 'мой', 'наш', 'ваш', 'их', 'его', 'её', 'их',
    'этот', 'тот', 'где', 'который', 'либо', 'нибудь', 'нет', 'да'
]

punctuation_signs = [',', '.']

In [5]:
def morph_parse(w):
    # TODO: вероятно не разбирает ещё какие-то части речи
    grammars = {
        'ADJF': 'ADJ',
        'ADVB': 'ADV',
        'INFN': 'VERB',
    }
    p = morph.parse(w)
    try:
        p = max(p, key=lambda x: (x.score, x.methods_stack[0][2]))  # взято из статьи на habr
    except Exception:
        p = p[0]
    return p.normal_form, '_' + grammars.get(p.tag.POS, p.tag.POS) if p.tag.POS else None

def get_clean_rusvectores_words(text):
    words = []
    text = text.replace('.', '. ')
    sentences = nltk.sent_tokenize(text, language="russian")
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence, language="russian")
        for t in tokens:
            w, POS = morph_parse(t.strip(',.')) #
            if not POS:  # TODO: просто пропускаем слова без части речи
                continue
            if w and w not in stop_words and w not in punctuation_signs:
                words.append(w+POS)
    return words

In [6]:
from sklearn.metrics import pairwise_distances

def get_distance_matrix_for_corpus(corpus):
    """ """
    X = [[i] for i in range(0, len(corpus))]
    
    distance = lambda x, y: model.wmdistance(corpus[int(x[0])]["cleaned_rusvectores_words"], corpus[int(y[0])]["cleaned_rusvectores_words"])
    
    return pairwise_distances(X, metric=distance, n_jobs=-1)

In [66]:
import csv
import nltk

def read_corpus(file_path):
    """ Читает корпус начального текста """
    result = []

    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # skip header
        for row in csv_reader:
            orig_text = row[1]
            cleaned_rusvectores_words = get_clean_rusvectores_words(orig_text)
            result.append(
                dict(
                    id=row[0],
                    orig_text=orig_text,
                    category=row[2],
                    theme=row[3],
                    executor=row[4],
                    cleaned_rusvectores_words=cleaned_rusvectores_words
                )
            )

    return result

corpus = read_corpus("../input/NashDomRyazan-29-03-2019.csv")[0:100]

In [48]:
d = get_distance_matrix_for_corpus(corpus)

In [49]:
targets = []
categories = dict()  # name -> index
for row in corpus:
    if row["category"] not in categories:
        categories[row["category"]] = len(categories)
    targets.append(categories[row["category"]])

In [50]:
categories

{'Дворовая территория': 0,
 'Парковки': 1,
 'Дороги': 2,
 'Городская территория': 3,
 'Остановки общественного транспорта': 4,
 'Государственное управление и местное самоуправление': 5,
 'Образование (школы, детские сады)': 6,
 'Многоквартирные дома': 7,
 'Бизнес': 8,
 'Природные ресурсы и охрана окружающей среды': 9,
 'ЖКХ': 10}

In [64]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier(n_neighbors=2, metric="precomputed")

knc.fit(d, targets)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='precomputed',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [65]:
text = "Разбитая (отсутствуют стёкла) ООТ \"Улица Пушкина\" в сторону Центра напротив дома ул. Ленинского Комсомола, 19. Просьба восстановить остекление."

test_vectors = get_clean_rusvectores_words(text)

test_distances = [model.wmdistance(test_vectors, c["cleaned_rusvectores_words"]) for c in corpus]

result = knc.predict_proba([test_distances])

result

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

[0,
 1,
 2,
 3,
 3,
 4,
 2,
 2,
 4,
 0,
 2,
 2,
 3,
 2,
 0,
 4,
 2,
 4,
 5,
 0,
 3,
 0,
 3,
 6,
 1,
 2,
 2,
 0,
 7,
 5,
 7,
 2,
 7,
 5,
 0,
 2,
 0,
 7,
 3,
 3,
 3,
 2,
 2,
 0,
 7,
 0,
 7,
 8,
 0,
 0,
 7,
 7,
 7,
 4,
 7,
 7,
 7,
 2,
 0,
 2,
 0,
 9,
 7,
 7,
 7,
 2,
 7,
 9,
 7,
 7,
 7,
 10,
 10,
 10,
 2,
 9,
 9,
 7,
 2,
 2,
 0,
 10,
 2,
 2,
 0,
 7,
 7,
 10,
 10,
 7,
 0,
 2,
 2,
 0,
 3,
 7,
 0,
 0,
 0,
 2]